In [29]:
import pandas as pd
import seaborn as sns
from numpy import nan, mean
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [32]:
train_data = pd.read_csv("./titanic/train.csv")
scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(train_data.Cabin.values.reshape(-1, 1))


Y = train_data.Survived
X = train_data.drop(columns=["PassengerId", "Survived", "Name", "Ticket", "Cabin", "Embarked"], inplace=False)

X.Sex = X.Sex.map({"male": 1, "female": 2})
X.Age.fillna(X.Age.mean(), inplace=True)

new_cabin_features = encoder.transform(train_data.Cabin.values.reshape(-1, 1))
new_cabin = pd.DataFrame(new_cabin_features, columns=[f"Cabin-{str(i)}" for i in range(new_cabin_features.shape[1])])
X = pd.concat([X, new_cabin], axis=1)

X = scaler.fit_transform(X)

In [33]:
test_features = pd.read_csv("./titanic/test.csv")
test_results = pd.read_csv("./titanic/gender_submission.csv")
test_data = pd.merge(
    test_features,
    test_results,
    on=["PassengerId"],
    how="inner"
)

Yt = test_data.Survived
Xt = test_data.drop(columns=["PassengerId", "Survived", "Name", "Ticket", "Cabin", "Embarked"], inplace=False)

Xt.Sex = Xt.Sex.map({"male": 1, "female": 2})
Xt.Age.fillna(Xt.Age.mean(), inplace=True)
Xt.Fare.fillna(Xt.Fare.mean(), inplace=True)

new_cabin_features = encoder.transform(test_data.Cabin.values.reshape(-1, 1))
new_cabin = pd.DataFrame(new_cabin_features, columns=[f"Cabin-{str(i)}" for i in range(new_cabin_features.shape[1])])
Xt = pd.concat([Xt, new_cabin], axis=1)

Xt = scaler.fit_transform(Xt)

In [34]:
KNN = KNeighborsClassifier(n_neighbors=30)
KNN.fit(X, Y)

Yp = KNN.predict(Xt)
print(f"KNN: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"KNN: Правильность на тестовом наборе: {KNN.score(Xt, Yt)}")
print(f"KNN: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"KNN: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

KNN: Правильность на тестовом наборе: 0.9641148325358851
KNN: Правильность на тестовом наборе: 0.9641148325358851
KNN: САО на тестовом наборе: 0.03588516746411483
KNN: СКО на тестовом наборе: 0.03588516746411483


In [95]:
DT = DecisionTreeClassifier(
    max_depth=1
)
DT.fit(X, Y)

Yp = DT.predict(Xt)
print(f"DT: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"DT: Правильность на тестовом наборе: {DT.score(Xt, Yt)}")
print(f"DT: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"DT: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

DT: Правильность на тестовом наборе: 1.0
DT: Правильность на тестовом наборе: 1.0
DT: САО на тестовом наборе: 0.0
DT: СКО на тестовом наборе: 0.0


In [87]:
RF = RandomForestClassifier(
    n_estimators=240,
    max_depth=17,
    random_state=1
)
RF.fit(X, Y)

Yp = RF.predict(Xt)
print(f"RF: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"RF: Правильность на тестовом наборе: {RF.score(Xt, Yt)}")
print(f"RF: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"RF: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

RF: Правильность на тестовом наборе: 0.8755980861244019
RF: Правильность на тестовом наборе: 0.8755980861244019
RF: САО на тестовом наборе: 0.12440191387559808
RF: СКО на тестовом наборе: 0.12440191387559808


In [85]:
AB = AdaBoostClassifier(n_estimators=1, learning_rate=1)
AB.fit(X, Y)

Yp = AB.predict(Xt)
print(f"AB: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"AB: Правильность на тестовом наборе: {AB.score(Xt, Yt)}")
print(f"AB: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"AB: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

AB: Правильность на тестовом наборе: 1.0
AB: Правильность на тестовом наборе: 1.0
AB: САО на тестовом наборе: 0.0
AB: СКО на тестовом наборе: 0.0


In [117]:
GNB = GaussianNB(var_smoothing=6)
GNB.fit(X, Y)

Yp = GNB.predict(Xt)
print(f"GNB: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"GNB: Правильность на тестовом наборе: {GNB.score(Xt, Yt)}")
print(f"GNB: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"GNB: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

GNB: Правильность на тестовом наборе: 0.6555023923444976
GNB: Правильность на тестовом наборе: 0.6555023923444976
GNB: САО на тестовом наборе: 0.3444976076555024
GNB: СКО на тестовом наборе: 0.3444976076555024


In [106]:
LR = LogisticRegression() 
LR.fit(X, Y)

Yp = LR.predict(Xt)
print(f"LR: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"LR: Правильность на тестовом наборе: {LR.score(Xt, Yt)}")
print(f"LR: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"LR: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

LR: Правильность на тестовом наборе: 0.8923444976076556
LR: Правильность на тестовом наборе: 0.8923444976076556
LR: САО на тестовом наборе: 0.1076555023923445
LR: СКО на тестовом наборе: 0.1076555023923445


In [73]:
SGD = SGDClassifier(
    loss="modified_huber",
    alpha=0.21,
    random_state=1
)
SGD.fit(X, Y)

Yp = SGD.predict(Xt)
print(f"SGD: Правильность на тестовом наборе: {mean(Yp == Yt)}")
print(f"SGD: Правильность на тестовом наборе: {SGD.score(Xt, Yt)}")
print(f"SGD: САО на тестовом наборе: {mean_absolute_error(Yt, Yp)}")
print(f"SGD: СКО на тестовом наборе: {mean_squared_error(Yt, Yp)}")

SGD: Правильность на тестовом наборе: 0.9593301435406698
SGD: Правильность на тестовом наборе: 0.9593301435406698
SGD: САО на тестовом наборе: 0.04066985645933014
SGD: СКО на тестовом наборе: 0.04066985645933014
